## n_job for grid searchCV & randomforest!

In [5]:
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler as zscore # zscore
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso #LRlasso
from collections import OrderedDict
from joblib import dump, load #to save models in files
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import re
import json
import os

In [6]:
# smoll change

In [23]:
from sklearn.model_selection import GridSearchCV
def gridcv(X, y, model, param_grid, naimpute=False, prepy=True, scorer = 'neg_mean_squared_error', cv_meth = LeaveOneOut(), cv_n_jobs = 1):
    """
    Perform Cross-Validation (defaukt: LOOCV) with hyperparameter tuning using GridSearchCV.
    
    Parameters:
    ----------
    X : pandas DataFrame or numpy array
        The feature matrix.
        
    y : pandas Series or numpy array
        The target variable.
        
    model : scikit-learn estimator
        The machine learning model to be used, should be an uninitialized model instance 
        (e.g., Lasso(), not Lasso(alpha=1.0)).
        
    param_grid : dict
        Dictionary containing the hyperparameters to be tuned and their possible values. 
        The keys should be prefixed with 'regressor__' to work with the pipeline.
        
    naimpute : bool, optional (default=False)
        Toggle imputation for missing values. 
        Currently not implemented; will print a message and return 0 if set to True.
        
    prepy : bool, optional (default=True)
        Toggle preprocessing target variable 'y' by setting any negative values to zero.
        
    scorer : str, callable, or None, optional (default='neg_mean_squared_error')
        A string or a scorer callable object / function with signature scorer(estimator, X, y). 
        For valid scoring strings, see the scikit-learn documentation.
        
    cv_meth : cross-validation generator, optional (default=LeaveOneOut())
        A cross-validation splitting strategy. 
        Possible inputs for cv are integers to specify the number of folds in a (Stratified)KFold, 
        CV splitter, cross-validation generator iterators, or an iterable.
        
    Returns:
    -------
    overall_metric : dict
        Dictionary containing the overall metrics and other details from the GridSearchCV.
        
    out_model : GridSearchCV object
        Fitted GridSearchCV object.
        
    best_params : dict
        Dictionary containing the best hyperparameters found by GridSearchCV.

    Call:
    ------
    from sklearn.model_selection import KFold

    # set up KFold cross-validator
    kfold_cv = KFold(n_splits=3, shuffle=True, random_state=42)

    param_grid = {
        'regressor__alpha': np.array(np.arange(0.0125, 0.0425, 0.0025)),
        'regressor__fit_intercept': [True, False]
    }
    print(param_grid)

    # Call the gridcv function with KFold as the cross-validation method
    lasso_fullkfold_scores, lasso_fullkfold_model, best_param = gridcv(
        X, 
        y,
        Lasso(max_iter=4000),
        param_grid,
        scorer='r2', 
        cv_meth=kfold_cv
    )
    dump(lasso_fullkfold_model, './models/lasso_fullkfold_model.pkl') # save the model as .pkl
    """

    # overall_metric = {'CV': cv_meth, 'scoring_metric': scorer} originally
    overall_metric = {'CV': str(cv_meth), 'scoring_metric': str(scorer)} # transformed to string because json dump scores later

    if prepy:
        y[y < 0] = 0
    
    if naimpute:
      print("not implemented")
      return 0


    pipeline = Pipeline([
        ('scaler', zscore()), 
        ('regressor', model)        # Regression model
    ])

    
    # declaring an Grid object
    # score : https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    out_model = GridSearchCV(pipeline, param_grid=param_grid, cv=cv_meth, scoring=scorer, n_jobs=cv_n_jobs).fit(X,y)
    # GridSearchCV need the regressor__ prefix for the pipiline object in the para_grid later when called

    best_pipeline = out_model.best_estimator_
    y_pred = best_pipeline.predict(X)

    overall_metric['correlation_true_pred'] = list(np.corrcoef(list(y), list(y_pred)))
    overall_metric['correlation_true_pred'][0] = list(overall_metric['correlation_true_pred'][0])
    overall_metric['correlation_true_pred'][1] = list(overall_metric['correlation_true_pred'][1])


    # LOOCV folds: split{i}_test_score (number of data points minus one) 
    overall_metric['fold_scores'] = [out_model.cv_results_[f'split{i}_test_score'][out_model.best_index_] for i in range(out_model.n_splits_)]
    best_params = out_model.best_params_


    # printing section
    print("best parameter from gridsearch>>\n", out_model.best_params_)
    print(overall_metric['CV'])
    print(overall_metric['scoring_metric'])
    print("correlation Matrix>>\n", overall_metric['correlation_true_pred'])
    print("scores for each fold>>\n",overall_metric['fold_scores'])

    if str(model).startswith("Lasso"):
        # access the 'regressor' step from the best pipeline and then its coefficients
        coefficients = best_pipeline.named_steps['regressor'].coef_
        overall_metric['non_zero_coefficients'] = coefficients[coefficients != 0]
        overall_metric['non_zero_coefficients'] = overall_metric['non_zero_coefficients'].tolist()
        overall_metric['non_zero_features'] = list(X.columns[np.where(coefficients != 0)[0]])
        print("non_zero_features>>\n",overall_metric['non_zero_features'])

    if str(model).startswith("RandomForestRegressor"):
        pass
        #feature_names = X.columns
        #feature_importances = best_pipeline.named_steps['regressor'].feature_importances_
        #feature_importance_dict = dict(zip(feature_names, feature_importances))
        #sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
        #overall_metric['feature_importances'] = OrderedDict(sorted_feature_importance)
        #print("feature_importances>>\n",overall_metric['feature_importances'])
       

    return overall_metric, out_model, best_params


In [8]:
def nX_cross_validation2(X, target, param_grid, scorer_estimate, output_prefix, random_states, output_path='./models/10xKfold/', n_splits=3):
    if os.path.exists(output_path):
        print(f"The path {output_path} exists.")
    else:
        print(f"The path {output_path} does not exist.")
        raise FileNotFoundError(f"The path {output_path} does not exist.")

    cv_results = {'random_state': [], 'scores': {}, 'mean_scores': [], 'common_features': {}, 'model': {}}
    for ran_state in random_states:
        print(ran_state)
        kfold_cv = KFold(n_splits=n_splits, shuffle=True, random_state=ran_state)
        scores, model, best_param = gridcv(
            X, 
            target,
            Lasso(),
            param_grid,
            prepy=False,
            scorer=scorer_estimate, 
            cv_meth=kfold_cv
        )
        cv_results['random_state'].append(ran_state)
        cv_results['scores'][ran_state] = scores
        cv_results['mean_scores'].append(np.mean(scores['fold_scores']))
        # cv_results['model'][ran_state] = model

    # Determine common features...
    cv_results['common_features'] = set(cv_results['scores'][42]['non_zero_features'])
    for r in cv_results['random_state'][1:]:
        current_features = set(cv_results['scores'][r]['non_zero_features'])
        cv_results['common_features'] = cv_results['common_features'].intersection(current_features)
    cv_results['common_features'] = list(cv_results['common_features'])

    #save to json
    with open(f"{output_path}{output_prefix}_nXcv.json", 'w') as file:
       json.dump(cv_results, file)
    file.close()

    return cv_results




### REF 1
        #if np.mean(scores['fold_scores']) > 0.3:
        #    print(f"\n >> TRUE, mean fold scores {np.mean(scores['fold_scores'])} is bigger than tresh << \n")
            # select feature based on cumulative importance
        #    cumulative_importance = 0.0
        #   selected_features = []
        #    for feature, importance in scores['feature_importances'].items():
        #        cumulative_importance += importance
        #        selected_features.append(feature)
        #        if cumulative_importance >= 0.95:
        #           break
        #    cv_results['selected_features'][ran_state] = selected_features
        # cv_results['model'][ran_state] = model

    
    # Determine common features selected on cumulative importance
    #first_key = list(cv_results['selected_features'])[0]
    #cv_results['common_features'] = set(cv_results['selected_features'][first_key])

    #for r in list(cv_results['selected_features'].keys())[1:]:
    #    current_features = set(cv_results['selected_features'][r])
    #    cv_results['common_features'] = cv_results['common_features'].intersection(current_features)
    #cv_results['common_features'] = list(cv_results['common_features'])

In [9]:
def convert_type(obj):
    """
    Converts an type of numpy into a python inherent type
    This function can be used in combination with json.dump:
    ----
    Usage Example:
    
    with open(f"{output_path}{output_prefix}_nXcv.json", 'w') as file:
       json.dump(cv_results, file, default=convert_type)
    """
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

In [24]:
def nX_cross_validation(X, target, param_grid, scorer_estimate, output_prefix, random_states, output_path='./models/10xKfold/', n_splits=3, cv_n_jobs=1, regr_n_job=1):
    if os.path.exists(output_path):
        print(f"The path {output_path} exists.")
    else:
        print(f"The path {output_path} does not exist.")
        raise FileNotFoundError(f"The path {output_path} does not exist.")
    best_fold_mean = float('-inf')
    best_model = []

    #cv_results = {'random_state': [], 'scores': {}, 'mean_scores': [], 'selected_features': {}, 'common_features': {}, 'model': {}}
    cv_results = {'random_state': [], 'scores': {}, 'mean_scores': [], 'selected_features': {}, 'best_param': []}

    for ran_state in random_states:
        print(ran_state)
        kfold_cv = KFold(n_splits=n_splits, shuffle=True, random_state=ran_state)
        scores, model, best_param = gridcv(
            X, 
            target,
            RandomForestRegressor(n_jobs=regr_n_job),
            param_grid,
            prepy=False,
            scorer=scorer_estimate, 
            cv_meth=kfold_cv,
            cv_n_jobs=cv_n_jobs
        )
        cv_results['random_state'].append(ran_state)
        cv_results['scores'][ran_state] = scores
        cv_results['mean_scores'].append(np.mean(scores['fold_scores']))
        if best_fold_mean < np.mean(scores['fold_scores']):
            best_fold_mean = np.mean(scores['fold_scores'])
            cv_results['best_param'] = best_param, ran_state, np.mean(scores['fold_scores'])

    # REF 1

    print(f"best estimator>>\n found in split: {cv_results['best_param'][1]}\n param_grid: {cv_results['best_param'][0]}\n mean fold score {cv_results['best_param'][2]}")    
    regr = RandomForestRegressor(max_features=cv_results['best_param'][0]['regressor__max_features'], n_estimators=cv_results['best_param'][0]['regressor__n_estimators'], bootstrap=cv_results['best_param'][0]['regressor__bootstrap'], n_jobs=regr_n_job)
    best_model = regr.fit(X, target)
    feature_names = X.columns
    feature_importances = best_model.feature_importances_
    feature_importance_dict = dict(zip(feature_names, feature_importances))
    sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
    sorted_feature_importance = OrderedDict(sorted_feature_importance)
    # select feature based on cumulative importance
    cumulative_importance = 0.0
    selected_features = []
    for feature, importance in sorted_feature_importance.items():
        print(f"feaute, {feature},  import, {importance}")
        cumulative_importance += importance
        selected_features.append(feature)
        if cumulative_importance >= 0.95:
            break
    cv_results['selected_features'] = selected_features

    #save to json
    with open(f"{output_path}{output_prefix}_nXcv.json", 'w') as file:
       json.dump(cv_results, file, default=convert_type)
    file.close()
    
    return cv_results




In [11]:
def to_valid_variable_name(name):
    # Replace special characters with underscores
    name = re.sub(r'\W|^(?=\d)', '_', name)
    # Reduce multiple consecutive underscores to one
    name = re.sub(r'_{2,}', '_', name)
    # Truncate length if necessary
    max_length = 30
    if len(name) > max_length:
        name = name[:max_length]
    # Ensure it doesn't start with a number
    if name[0].isdigit():
        name = "_" + name
    return name

In [12]:
tr_mut = pd.read_csv("/home/t44p/PW_rawdata/tr_gc_mutual/tr_mut.csv", sep=",")
gcms_mut = pd.read_csv("/home/t44p/PW_rawdata/tr_gc_mutual/gcms_mut.csv", sep=",")
lcms_mut = pd.read_csv("/home/t44p/PW_rawdata/tr_gc_mutual/lcms_mut.csv", sep=",")

X = pd.read_csv("/home/t44p/PW_rawdata/tr_gc_mutual/tr_mut_transposed.csv", sep=",")
#

In [18]:
"""from sklearn.datasets import make_regression
X, y = make_regression(n_features=4, n_informative=2,
                       random_state=0, shuffle=False)"""
regr = RandomForestRegressor(max_depth=2, random_state=0, n_jobs=3)
regr.fit(X.iloc[:,:], gcms_mut.iloc[59,1:])
print(regr.predict(X.iloc[:,:]))

[0.06115402 0.19094228 0.26527444 0.29463637 0.32083137 0.33420758
 0.04894513 0.22726775 0.31350062 0.3055881  0.27004845 0.35836926
 0.40416474 0.05106125 0.2216727  0.29773878 0.31415813 0.32469304
 0.40883514 0.36435307 0.21304547 0.45176721 0.42762626 0.41732094
 0.38741296 0.3525298  0.30096443 0.37433194 0.43484489 0.34584996
 0.41922279 0.321271   0.42421353 0.44178561 0.42598215 0.35318408
 0.31890495 0.30005904 0.36642328 0.44253155]


In [19]:
kfold_cv = KFold(n_splits=3, shuffle=True, random_state=42)

param_grid = {
    'regressor__alpha': np.array(np.arange(0.0125, 0.0425, 0.0025)),
    'regressor__fit_intercept': [True, False]
}   
lasso_fullkfold_scores, lasso_fullkfold_model, lasso_best_param = gridcv(
    X.iloc[:,0:10], 
    gcms_mut.iloc[59,1:],
    Lasso(max_iter=4000),
    param_grid,
    scorer='r2', 
    cv_meth=kfold_cv
)

best parameter from gridsearch>>
 {'regressor__alpha': 0.0175, 'regressor__fit_intercept': True}
KFold(n_splits=3, random_state=42, shuffle=True)
r2
correlation Matrix>>
 [[1.0, 0.7481332260894714], [0.7481332260894714, 0.9999999999999998]]
scores for each fold>>
 [0.7720139897938894, 0.3313252228931619, 0.25279959004526065]
non_zero_features>>
 ['Xele.ptg000045l.1', 'Xele.ptg000045l.6', 'Xele.ptg000045l.8', 'Xele.ptg000045l.10']


In [69]:
feature_names = X.columns
feature_importances = regr.feature_importances_
feature_importance_dict = dict(zip(feature_names, feature_importances))

# sort this dictionary based on importances
sorted_feature_importance = OrderedDict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True))

print(type(sorted_feature_importance))


<class 'collections.OrderedDict'>


In [37]:
#np.log10(np.array(np.arange(200, 1401, 600)))#
np.round(np.exp2(np.array(np.arange(7.2, 15.3, 3)))).astype(int)

array([ 147, 1176, 9410])

# the run below took 16 min on the full X

>explore a better grid on the cluster 

In [15]:
print(f"'regressor__n_estimators': {np.array(np.arange(500, 1501, 200))}")
print(f"'regressor__max_features':{np.round(np.exp2(np.array(np.arange(3.2, 13.3, 2)))).astype(int)}")
print(f"'regressor__n_estimators': {np.array(np.arange(10, 15, 1))}")
print(f"'regressor__max_features':{np.array(np.arange(7, 10, 1))}")


'regressor__n_estimators': [ 500  700  900 1100 1300 1500]
'regressor__max_features':[   9   37  147  588 2353 9410]
'regressor__n_estimators': [10 11 12 13 14]
'regressor__max_features':[7 8 9]


In [20]:
kfold_cv = KFold(n_splits=3, shuffle=True, random_state=42)

param_grid = {
    'regressor__n_estimators': np.array(np.arange(500, 1501, 1000)),
    'regressor__max_features': np.round(np.exp2(np.array(np.arange(7.2, 15.3, 3)))).astype(int),
    'regressor__bootstrap': [False, True]
}   
rfr_fullkfold_scores, rfr_fullkfold_model, rfr_best_param = gridcv(
    X.iloc[:,:100], 
    gcms_mut.iloc[59,1:],
    RandomForestRegressor(n_jobs=2),
    param_grid,
    scorer='r2', 
    cv_meth=kfold_cv,
    cv_n_jobs=2
)
#for key, value in rfr_fullkfold_scores.items():
#    print(f"{key} >>>>\n {value}\n\n")

best parameter from gridsearch>>
 {'regressor__bootstrap': True, 'regressor__max_features': 1176, 'regressor__n_estimators': 500}
KFold(n_splits=3, random_state=42, shuffle=True)
r2
correlation Matrix>>
 [[1.0, 0.9752316306083173], [0.9752316306083174, 1.0]]
scores for each fold>>
 [0.5069292268821795, 0.6047388361149414, 0.2653775032594198]
feature_importances>>
 OrderedDict([('Xele.ptg000045l.50', 0.41926256928952893), ('Xele.ptg000045l.1', 0.05245621017973865), ('Xele.ptg000045l.18', 0.03649779466045334), ('Xele.ptg000045l.15', 0.03332675667963292), ('Xele.ptg000045l.93', 0.0328994032125989), ('Xele.ptg000045l.100', 0.030150458190898618), ('Xele.ptg000045l.98', 0.020536180125984703), ('Xele.ptg000045l.108', 0.015845226685376773), ('Xele.ptg000045l.101', 0.015688698118336536), ('Xele.ptg000045l.59', 0.013549898156599452), ('Xele.ptg000045l.74', 0.011384589630555634), ('Xele.ptg000045l.19', 0.01047043602816892), ('Xele.ptg000045l.103', 0.01044245988667525), ('Xele.ptg000045l.87', 0.00

In [25]:
print(f"RFR START >>> {to_valid_variable_name(str(gcms_mut.iloc[59,0]))}\n\n")
tenX = [42, 43, 44]#, 45, 46, 47, 48, 49, 50, 51, 52]
out = './py/10xKfold/test_rfr/'
param_grid = {
    'regressor__n_estimators': np.array(np.arange(10, 15, 1)),
    'regressor__max_features': np.array(np.arange(7, 10, 1)),
    'regressor__bootstrap': [False, True]
}   
sucrose_10xKfold = nX_cross_validation(X.iloc[:,:100], gcms_mut.iloc[59,1:], param_grid, 'r2', to_valid_variable_name(str(gcms_mut.iloc[59,0])), tenX, output_path=out, cv_n_jobs=2, regr_n_job=2)


RFR START >>> sucrose_437_361_rt13_77


The path ./py/10xKfold/test_rfr/ exists.
42
best parameter from gridsearch>>
 {'regressor__bootstrap': True, 'regressor__max_features': 9, 'regressor__n_estimators': 13}
KFold(n_splits=3, random_state=42, shuffle=True)
r2
correlation Matrix>>
 [[1.0, 0.9704156298966722], [0.9704156298966721, 0.9999999999999999]]
scores for each fold>>
 [0.5411879858317737, 0.42902222689265, 0.2822698944408427]
43
best parameter from gridsearch>>
 {'regressor__bootstrap': False, 'regressor__max_features': 8, 'regressor__n_estimators': 12}
KFold(n_splits=3, random_state=43, shuffle=True)
r2
correlation Matrix>>
 [[1.0, 0.9999999999999998], [0.9999999999999998, 1.0]]
scores for each fold>>
 [0.3675948428691159, 0.5552435547838133, 0.4849802299742626]
44
best parameter from gridsearch>>
 {'regressor__bootstrap': True, 'regressor__max_features': 9, 'regressor__n_estimators': 14}
KFold(n_splits=3, random_state=44, shuffle=True)
r2
correlation Matrix>>
 [[1.0, 0.9821567

In [16]:
#for ele in sucrose_10xKfold['selected_features']:
#    print(f"{sucrose_10xKfold['selected_features'][ele]}")
sucrose_10xKfold['mean_scores']

sucrose_10xKfold.keys()


sucrose_10xKfold['selected_features']

['Xele.ptg000045l.101',
 'Xele.ptg000045l.50',
 'Xele.ptg000045l.28',
 'Xele.ptg000045l.1',
 'Xele.ptg000045l.107',
 'Xele.ptg000045l.89',
 'Xele.ptg000045l.21',
 'Xele.ptg000045l.25',
 'Xele.ptg000045l.93',
 'Xele.ptg000045l.98',
 'Xele.ptg000045l.6',
 'Xele.ptg000045l.87',
 'Xele.ptg000045l.16',
 'Xele.ptg000045l.7',
 'Xele.ptg000045l.24',
 'Xele.ptg000045l.56',
 'Xele.ptg000045l.74',
 'Xele.ptg000045l.78',
 'Xele.ptg000045l.94',
 'Xele.ptg000045l.84',
 'Xele.ptg000045l.41',
 'Xele.ptg000045l.69',
 'Xele.ptg000045l.68',
 'Xele.ptg000045l.108',
 'Xele.ptg000045l.20',
 'Xele.ptg000045l.19',
 'Xele.ptg000045l.102',
 'Xele.ptg000045l.54',
 'Xele.ptg000045l.15',
 'Xele.ptg000045l.38',
 'Xele.ptg000045l.92',
 'Xele.ptg000045l.100',
 'Xele.ptg000045l.73',
 'Xele.ptg000045l.33',
 'Xele.ptg000045l.58',
 'Xele.ptg000045l.36',
 'Xele.ptg000045l.26',
 'Xele.ptg000045l.42',
 'Xele.ptg000045l.55',
 'Xele.ptg000045l.112',
 'Xele.ptg000045l.76',
 'Xele.ptg000045l.65',
 'Xele.ptg000045l.44',
 'Xele.p

In [105]:
med_sel_feets = []
for key, val in rfr_fullkfold_scores['feature_importances'].items():
    if val >= np.median(list(rfr_fullkfold_scores['feature_importances'].values())):
        med_sel_feets.append(key)


cumulative_importance = 0.0
selected_features = []

for feature, importance in rfr_fullkfold_scores['feature_importances'].items():
    cumulative_importance += importance
    selected_features.append(feature)
    if cumulative_importance >= 0.95:
        break

print(f"features >>\n {len(rfr_fullkfold_scores['feature_importances'].keys())}\nmedian >> \n {np.median(list(rfr_fullkfold_scores['feature_importances'].values()))}\n med_sel_feat>>\n{len(med_sel_feets)}\nselected features >> \n{len(selected_features)}")

features >>
 1000
median >> 
 0.0
 med_sel_feat>>
1000
selected features >> 
62


In [139]:
list(rfr_fullkfold_scores['feature_importances'])[0]

'Xele.ptg000045l.169'